LangChain Ollama integration

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

# template = """Question: {question}

# Answer: Let's think step by step."""

# prompt = ChatPromptTemplate.from_template(template)

# llm = OllamaLLM(model="llama3.1:8b")

# chain = prompt | llm

# chain.invoke({"question": "What is LangChain?"})

LangChain Ollama integration - Chat interface

In [2]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.1:8b")

# messages = [
#     ("system", "You are a helpful assistant answering user queries."),
#     ("human", "What is LangChain?"),
# ]
# llm.invoke(messages)

Embeddings


In [3]:
# from langchain_ollama import OllamaEmbeddings

# embeddings = OllamaEmbeddings(model="llama3.1:8b")

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/Users/david/miniconda3/envs/hf_gpu/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/david/miniconda3/envs/hf_gpu/lib/python3.11/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/david/miniconda3/envs/hf_gpu/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <BFAC0362-079C-3D60-A731-79532A75BA60> /Users/david/miniconda3/envs/hf_gpu/lib/python3.11/site-packages/torchvision/image.so
  Reason: tried: '/Users/david/miniconda3/envs/hf_gpu/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/david/miniconda3/envs/hf_gpu/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/david/miniconda3/envs/hf_gpu/

In [5]:
# embeddings = HuggingFaceEmbeddings(model_name='jinaai/jina-embeddings-v2-base-de')

In Memory database

In [6]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

Load PDF Documents

In [7]:
from langchain_community.document_loaders import PyMuPDFLoader

file_path = "./pubhub_docs/Warum Lachen gesund ist_DE.pdf"
loader = PyMuPDFLoader(file_path,
        mode="single",
    )

In [8]:
docs = loader.load()
print(len(docs))
docs[0].metadata

1


{'producer': 'macOS Version 15.3.2 (Build 24D81) Quartz PDFContext',
 'creator': 'Safari',
 'creationdate': "D:20250324210225Z00'00'",
 'source': './pubhub_docs/Warum Lachen gesund ist_DE.pdf',
 'file_path': './pubhub_docs/Warum Lachen gesund ist_DE.pdf',
 'total_pages': 4,
 'format': 'PDF 1.4',
 'title': 'Warum Lachen gesund ist',
 'author': 'David Massimo',
 'subject': '',
 'keywords': '',
 'moddate': "D:20250324210225Z00'00'",
 'trapped': '',
 'modDate': "D:20250324210225Z00'00'",
 'creationDate': "D:20250324210225Z00'00'"}

In [9]:
#
# Loading + Parsing files
#

from langchain_community.document_loaders import FileSystemBlobLoader
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import PyMuPDFParser

loader = GenericLoader(
    blob_loader=FileSystemBlobLoader(
        path="./pubhub_docs/",
        glob="*.pdf",
    ),
    # blob_parser=PyMuPDFParser(mode="single"),
    blob_parser=PyMuPDFParser(mode="single"),
)
docs = loader.load()
len(docs[5].page_content)
# docs[0].metadata

9119

Tokenization - Doc splitting

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 68 sub-documents.


Store splits

In [11]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

['adf4daf9-763d-4195-a94e-eb58fea7ff52', 'c2571eaf-98d6-455d-9fbb-12ebd436160a', '39f684ef-094c-4285-bc10-428b37b007d8']


Retrieval and Generation

In [12]:
# from langchain import hub

# prompt = hub.pull("rlm/rag-prompt")

# example_messages = prompt.invoke(
#     {"context": "(context goes here)", "question": "(question goes here)"}
# ).to_messages()

# assert len(example_messages) == 1
# print(example_messages[0].content)

In [13]:
# from langchain_core.prompts import PromptTemplate

# template = """You are an helpful chatbot to support workers at Public Value Technologies gmbh (also known as pub.tech). 
# Use the following pieces of context to answer the question at the end.
# If you don't know the answer, just say that you don't know, don't try to make up an answer.
# Always say "thanks for asking!" at the end of the answer.

# If the query is in German you reply in german, otherwise in english.
# {context}

# Question: {question}

# Helpful Answer:"""
# prompt = PromptTemplate.from_template(template)

In [14]:
from langgraph.graph import MessagesState, StateGraph

graph_builder = StateGraph(MessagesState)

from langchain_core.tools import tool


@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode


# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}


# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])


# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        "You are an helpful chatbot for question-answering tasks supporting workers at Public Value Technologies gmbh (also known as pub.tech)."        
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know." 
        "\n\n"
        f"{docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

In [15]:
# from langchain_core.documents import Document
# from typing_extensions import List, TypedDict


# class State(TypedDict):
#     question: str
#     context: List[Document]
#     answer: str

# # Nodes
# def retrieve(state: State):
#     retrieved_docs = vector_store.similarity_search(state["question"],k=6)
#     return {"context": retrieved_docs}


# def generate(state: State):    
#     docs_content = "\n\n".join(doc.page_content for doc in state["context"])
#     messages = prompt.invoke({"question": state["question"], "context": docs_content})
#     # print(messages)
#     response = llm.invoke(messages)
#     return {"answer": response}

# # Control flow of the program (langgrap)h
# from langgraph.graph import START, StateGraph

# graph_builder = StateGraph(State).add_sequence([retrieve, generate])
# graph_builder.add_edge(START, "retrieve")
# graph = graph_builder.compile()

In [16]:
from langgraph.graph import END
from langgraph.prebuilt import ToolNode, tools_condition

graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()

In [17]:
input_message = "What should I do when doing a businesss trip?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What should I do when doing a businesss trip?
================================== Ai Message ==================================
Tool Calls:
  retrieve (de7bc1d7-4d86-4399-9f95-2840e655826b)
 Call ID: de7bc1d7-4d86-4399-9f95-2840e655826b
  Args:
    query: business travel tips
================================= Tool Message =================================
Name: retrieve

Source: {'producer': 'macOS Version 15.3.2 (Build 24D81) Quartz PDFContext', 'creator': 'Safari', 'creationdate': "D:20250324210537Z00'00'", 'source': 'pubhub_docs/Business trips and travel expense guidelines.pdf', 'file_path': 'pubhub_docs/Business trips and travel expense guidelines.pdf', 'total_pages': 3, 'format': 'PDF 1.3', 'title': 'Business trips and travel expense guidelines', 'author': 'David Massimo', 'subject': '', 'keywords': '', 'moddate': "D:20250324210537Z00'00'", 'trapped': '', 'modDate': "D:20250324210537Z00'00'", 'creatio

In [19]:
import pprint
# result = graph.invoke({"question": "Was soll ich tun um urlaub zu buchen at pub.tech?"})
result = graph.invoke({"question": "What should I do when doing a businesss trip?"})

pprint.pp(f'Context: {result["context"]}\n\n')
pprint.pp(f'Answer: {result["answer"]}')

KeyError: 'context'

In [ ]:
import pprint

In [ ]:



Table with embedding from the docs

0 - 1 -> how a specific is being represented by that doc

clustering of those documents. 


table of users -> decribing the topics for each user, 


consumption data (trend analysis)
